In [29]:
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler

In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip')

In [31]:
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [32]:
df.drop('time_signature', axis = 1, inplace = True)
df = df[(df['genre'] != 'Soundtrack') & (df['genre'] != 'Movie')]
df = df[(df['duration_ms'] >= 90000) & (df['duration_ms'] <= 900000)]
df['genre'].replace("Children’s Music", "Children's Music", inplace = True)
top_songs = df[df['popularity'] >= 80]
top_scaled = top_songs.copy()
scaler = MinMaxScaler()
top_scaled['loudness_scaled'] = scaler.fit_transform(top_songs['loudness'].values.reshape(-1, 1))

In [33]:
top_scaled.to_csv('data/top_polar.csv.zip', compression='zip')

In [34]:
df_music_dummies = pd.concat([df , df['genre'].str.get_dummies()], axis = 1)
df_music_dummies['mode'] = df_music_dummies['mode'].factorize()[0]
df_music_dummies["popularity_score"] = df["popularity"].apply(popularity_score)
cols = ['genre', 'artist_name', 'track_id', 'key', 'popularity']
df_music_dummies.drop(cols, axis=1, inplace=True)
df_music_without_0_pop = df_music_dummies[df_music_dummies['popularity_score'] != 0]

In [35]:
df_final = df_music_without_0_pop.copy()
df_final['tempo'].apply(round)
df_final['duration_ms'] = round(df_final['duration_ms'] / 1000)
df_final = df_final.astype({'tempo': 'int', 'duration_ms': 'int'})

In [36]:
df_final.to_csv('data/music_ml.csv.zip', compression='zip')

In [37]:
df_final

,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,World,popularity_score
135,Be Without You - Kendu Mix,0.08300,0.724,246,0.689,0.000000,0.3040,-5.922,0,0.1350,...,0,1,0,0,0,0,0,0,0,4
136,Desperado,0.32300,0.685,186,0.610,0.000000,0.1020,-5.221,0,0.0439,...,0,1,0,0,0,0,0,0,0,4
137,Ice On My Baby (feat. Kevin Gates) - Remix,0.06750,0.762,200,0.520,0.000004,0.1140,-5.237,0,0.0959,...,0,1,0,0,0,0,0,0,0,4
138,Heaven Falls / Fall on Me,0.36000,0.563,241,0.366,0.002430,0.0955,-6.896,0,0.1210,...,0,1,0,0,0,0,0,0,0,4
139,Love Myself,0.59600,0.653,214,0.621,0.000000,0.0811,-5.721,0,0.0409,...,0,1,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Son Of Slide,0.00384,0.687,326,0.714,0.544000,0.0845,-10.626,1,0.0316,...,0,0,0,0,0,0,0,1,0,2
232721,Burning Fire,0.03290,0.785,282,0.683,0.000880,0.2370,-6.944,0,0.0337,...,0,0,0,0,0,0,0,1,0,2
232722,(I'm Your) Hoochie Coochie Man,0.90100,0.517,167,0.419,0.000000,0.0945,-8.282,1,0.1480,...,0,0,0,0,0,0,0,1,0,3
232723,With My Words,0.26200,0.745,222,0.704,0.000000,0.3330,-7.137,1,0.1460,...,0,0,0,0,0,0,0,1,0,3


In [38]:
df_final.shape

(205488, 37)